对于更多人的来说，快速跑通一个最简单的DEMO是更为普遍的需求， 而MCP的官方项目（https://github.com/modelcontextprotocol/python-sdk）说明绕了半天，调试并不顺利，并不利于快速体验上手。基于本人需求，选取了mcp中服务端和客户端使用http传输的方式，纯python代码，其它的不做探究。以下是整理后的操练教程：

## 安装mcp

In [ ]:
pip install mcp

## 定义一个mcp server

In [ ]:
from mcp.server.fastmcp import FastMCP
from datetime import datetime

# 有状态服务端（维护会话状态）
mcp = FastMCP("StatefulServer")

# 定义一个工具，返回当前日期时间格式化字符串
@mcp.tool()
def get_current_time() -> str:
    return datetime.now().isoformat()

# 以流式方式启动服务
if __name__ == "__main__":
    mcp.run(transport="streamable-http")

## 运行服务端


python server.py # 直接运行
输出：
(mcp) E:\Project\mcp>python server.py
[32mINFO[0m:     Started server process [[36m17748[0m]
[32mINFO[0m:     Waiting for application startup.
[05/23/25 12:11:07] INFO     StreamableHTTP session manager started                      streamable_http_manager.py:109
[32mINFO[0m:     Application startup complete.
[32mINFO[0m:     Uvicorn running on [1mhttp://127.0.0.1:8000[0m (Press CTRL+C to quit)


mcp dev server.py  # 开发模式
输出：
(mcp) E:\Project\mcp>mcp dev server.py --with-editable .
Starting MCP inspector...
⚙️ Proxy server listening on port 6277
🔍 MCP Inspector is up and running at http://127.0.0.1:6274 🚀
打开 Inspector http://127.0.0.1:6274，将看到WEB调试页面



mcp dev server.py --with pandas --with numpy # 加载科学计算
mcp dev server.py --with-editable . # 挂载本地编辑器

## 编写客户端

In [ ]:
# client.py
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession
import asyncio

async def main():
    # 连接到 MCP 服务端
    async with streamablehttp_client("http://127.0.0.1:8000/mcp") as (
        read_stream,
        write_stream,
        _,
    ):
        # 创建客户端会话
        async with ClientSession(read_stream, write_stream) as session:
            # 初始化连接（必须）
            await session.initialize()

            # 调用服务端的 get_current_time 工具（无参数）
            tool_result = await session.call_tool("get_current_time", {})

            # print("当前时间：", tool_result)
            print("当前时间：", tool_result.content[0].text)


if __name__ == "__main__":
    asyncio.run(main())


## 运行客户端

python client.py

## 不同状态的服务端

In [ ]:
from mcp.server.fastmcp import FastMCP

# 有状态对话式服务、保留会话记忆
mcp = FastMCP("Stateful")
mcp.run(transport="streamable-http")

# 无状态 HTTP 工具服务（返回 SSE 流）
mcp = FastMCP("Stateless", stateless_http=True)
mcp.run(transport="streamable-http")

# 无状态 HTTP 工具服务（返回标准 JSON）
mcp = FastMCP("Stateless", stateless_http=True, json_response=True)
mcp.run(transport="streamable-http")

# transport="streamable-http" 使用流式输出

## 同时运行多个服务器

In [ ]:
# echo.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(name="EchoServer", stateless_http=True)


@mcp.tool(description="A simple echo tool")
def echo(message: str) -> str:
    return f"Echo: {message}"

In [ ]:
# math1.py 如果像官方教程文件名为math.py将会与python内置函数同名出错
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(name="MathServer", stateless_http=True)


@mcp.tool(description="A simple add tool")
def add_two(n: int) -> int:
    return n + 2

In [ ]:
# main.py # 对官方代码作了更正
import contextlib
from fastapi import FastAPI
import echo
import math1


# Create a combined lifespan to manage both session managers
@contextlib.asynccontextmanager
async def lifespan(app: FastAPI):
    async with contextlib.AsyncExitStack() as stack:
        await stack.enter_async_context(echo.mcp.session_manager.run())
        await stack.enter_async_context(math1.mcp.session_manager.run())
        yield


app = FastAPI(lifespan=lifespan)
app.mount("/echo", echo.mcp.streamable_http_app())
app.mount("/math", math1.mcp.streamable_http_app())

if __name__ == "__main__":
    import uvicorn
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)

执行：
python main.py